In [ ]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd

import random
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor

from keras import backend as K

# Chargement et correction des données

Le dataset utilisé contient les prix des actions des 81 principales compagnies du NASDAQ100. La valeur de l'index du NASDAQ est utilisé comme cible.  
La fréquence des information est d'une minute, depuis le 26 juillet 2016 jusqu'au 22 décembre 2016, soit 105 jours au total (les samedi et dimanche ne sont pas comptés, ainsi que le 25 novembre qui ne possède que 210 données et le 22 décembre qui n'en possède que 180).

**1. Chargement des données**

In [ ]:
!rm *.csv
!curl --location --remote-header-name --remote-name "https://github.com/AlexandreBourrieau/FICHIERS/raw/main/Series_Temporelles/Multi/Data/nasdaq100_padding.csv"

**2. Analyse et correction des données**

In [ ]:
# Création de la série sous Pandas
df_etude = pd.read_csv("nasdaq100_padding.csv",dtype="float32")
df_etude

Affiche les types :

In [ ]:
df_etude.dtypes

**5. Affiche les données**

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=np.linspace(0,len(df_etude),len(df_etude)+1),y=df_etude['NDX'], line=dict(color='blue', width=1),name="Index"))
fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

# Corrélations entre les variables

**1. Corrélation de l'ensemble des variables**

In [ ]:
import seaborn as sns

corr = df_etude.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=np.bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(15, 8))

sns.heatmap(corr,mask=mask, cmap='coolwarm',annot=True, fmt='.2f')

# Choix des paramètres de l'algorithme Random Forest

**1. Création des données X et Y**

In [ ]:
X = df_etude.iloc[:,0:-1]
X

In [ ]:
Y = df_etude.iloc[:,-1:]
Y

**2. Choix du nombre d'arbres**

Parmi les paramètres de l'algorithme [RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) nous trouvons :
 - **n_estimators** : C'est le nombre d'arbres utilisés
 - **bootstrap** : Choix d'utiliser ou non la méthode du bootstrap pour construire les arbres. Si le bootstrap n'est pas activé, tous les arbres seront constuits avec le même dataset.  
 - **oob_score** : Choix d'utiliser les échantillons out-of-bag pour calculer le score moyen des prédictions (uniquement utilisable si l'option bootstrap est activée)
 - **max_sample** : Si le bootstrap est activé, permet de choisir le nombre d'échantillons à prendre dans le dataset initial pour construire chaque estimateur.  
 - **max_features** : Le nombre de données prises aléatoirement pour découper découper un noeud de l'arbre (mtry)  

On commence par regarder comment évolue le score de l'OOB sur une 10ène d'essais avec un nombre d'arbres croissant, tout en gardant une valeur de mtry par défaut (racine carrée du nombre de variables en classification et nombre de variables divisé par 3 en régression). On choisira le nombre d'arbre tel qu'il soit minimal et que le score OOB soit stabilisé.  

In [ ]:
# Informations sur les données
n = 40560             # Nombre d'observations
p = 81                # Nombre de variables

n_arbres_max = 2000

n_arbres = np.linspace(50,n_arbres_max,10).astype(np.int32)
mtry = p/3
OOB_err = []

for i in n_arbres:
  print("#Arbres : %d" %i)
  clf = RandomForestRegressor(n_estimators=i, bootstrap=True, oob_score=True, max_samples = n, max_features = mtry, n_jobs=-1)
  clf.fit(X,tf.squeeze(np.asarray(Y),1))
  OOB_err.append(1 - clf.oob_score_)

In [ ]:
plt.plot(n_arbres,OOB_err)

**3. Choix du mtry**

On regarde comment évolue le score de l'OOB sur une 10ène d'essais avec un nombre d'arbres fixé à la valeur trouvée précédemment, puis en faisant varier la valeur de mtry de la moitié de sa valeur par défaut jusqu'à son maximum possible. On choisira la valeur de mtry telle qu'elle soit minimale et que le score OOB soit stabilisé.  

In [ ]:
# Informations sur les données
n = 40560             # Nombre d'observations
p = 81                # Nombre de variables

n_arbres = 2000

mtry_0 = np.int32(0.5*(p/3))

m_try = np.linspace(mtry_0,p,10).astype(np.int32)

OOB_err = []

for i in m_try:
   print("mtry = %s" %i)
   clf = RandomForestRegressor(n_estimators=n_arbres, bootstrap=True, oob_score=True, max_features=i, n_jobs=-1)
   clf.fit(X,tf.squeeze(np.asarray(Y),1))
   OOB_err.append(1 - clf.oob_score_)

In [ ]:
plt.plot(m_try,OOB_err)

On choisir n_arbres = 2000 et on utilise m_try = 20

# Importance des variables : Importance sans permutations (Gini Importance)

**1. Entrainement de la forêt**

In [ ]:
from sklearn.inspection import permutation_importance

# Informations sur les données
n = 40560             # Nombre d'observations
p = 81                # Nombre de variables
n_arbres = 2000
m_try = 20

clf = RandomForestRegressor(n_estimators=n_arbres, bootstrap=True, oob_score=True, max_features=m_try, n_jobs=-1)
clf.fit(X,tf.squeeze(np.asarray(Y),1))

**2. Affichage de l'importance des variables**

In [ ]:
col_sorted_by_importance=np.argsort(-clf.feature_importances_)

feat_imp = pd.DataFrame({'cols':X.columns[col_sorted_by_importance],'imps':clf.feature_importances_[col_sorted_by_importance]})
feat_imp

In [ ]:
!pip install plotly_express --upgrade -q

In [ ]:
import plotly_express as px
import plotly.offline as po

px.bar(feat_imp.sort_values(['imps'], ascending=False)[:30], x='cols', y='imps', labels={'cols':'column', 'imps':'feature importance'})

<img src="https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Series_Temporelles/Multi/images/NASDAQ_1.png?raw=true">

<img src="https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Series_Temporelles/Multi/images/NASDAQ_1.png?raw=true">

# Importance des variables - Méthode par permutations

Le calcul de l'importance par la méthode des permutations est une technique dans laquelle on mélange les valeurs d'une colonne afin d'observer l'impact sur le score obtenu. Si le score est beaucoup affecté, cela signifie que la variable permutée est très importante.

On utilise la méthode de Scikit-learn [permutation_importance](https://scikit-learn.org/stable/modules/generated/sklearn.inspection.permutation_importance.html#sklearn.inspection.permutation_importance)

In [ ]:
from sklearn.inspection import permutation_importance

# Informations sur les données
n = 40560             # Nombre d'observations
p = 81                # Nombre de variables
n_arbres = 2000
m_try = 20

clf = RandomForestRegressor(n_estimators=n_arbres, bootstrap=True, oob_score=True, max_features=m_try, n_jobs=1)
clf.fit(X,tf.squeeze(np.asarray(Y),1))
result = permutation_importance(clf, X, Y, n_repeats=10,n_jobs=1)

In [ ]:
result

In [ ]:
scores={c:[] for c in X.columns}
i=0
for c in scores:
  scores[c].append(result.importances_mean[i])
  i = i+1

In [ ]:
scores

In [ ]:
pd.DataFrame.from_dict(scores).melt().groupby(['variable']).mean().reset_index().sort_values(['value'], ascending=False)

In [ ]:
px.bar(
    pd.DataFrame.from_dict(scores).melt().groupby(['variable']).mean().reset_index().sort_values(['value'], ascending=False),
    x='variable',y='value',labels={'variable':'column','value':'% change in recall'})

<img src="https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Series_Temporelles/Multi/images/SML_PermImportance.png?raw=true">

# Comparaison entre les deux méthodes

In [ ]:
# Calcul des écarts-types et des moyennes
scores_std = {}
scores_mean = {}
for element in scores:
  scores_std[element] = np.std(scores[element])
  scores_mean[element] = np.mean(scores[element])

df_perm = pd.DataFrame.from_dict([scores_std, scores_mean]).transpose()
df_perm = df_perm.rename(columns={0:"std",1:"mean"})
df_perm = df_perm.sort_values(by=['mean'],ascending=False)
df_perm

In [ ]:
feature_perm = df_perm
feature_imp = feat_imp.sort_values(['imps'],ascending=False)
feature_imp = feature_imp

tree_indices = np.arange(0, len(feature_imp)) + 0.5

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8))

ax1.barh(tree_indices,feature_imp['imps'].values, height=0.7)
ax1.set_yticks(tree_indices)
ax1.set_yticklabels(feature_imp['cols'].values)
ax1.set_ylim((0, len(feature_imp)))

ax2.boxplot(feature_perm, vert=False,labels=feature_perm.index.values)
fig.tight_layout()
plt.show()

<img src="https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Series_Temporelles/Multi/images/NASDAQ_FEATURE3.png?raw=true">

# Sélection des variables par méthode RFE-CV

In [ ]:
from sklearn.feature_selection import RFECV

# Informations sur les données
n = 40560             # Nombre d'observations
p = 81                # Nombre de variables
n_arbres = 2000
m_try = p

min_features_to_select = 3

clf = RandomForestRegressor(n_estimators=n_arbres, bootstrap=True, oob_score=True, max_features="auto", n_jobs=-1)
rfecv = RFECV(estimator=clf, step=1, cv=5, scoring='neg_mean_absolute_error',min_features_to_select=min_features_to_select, verbose=1)
rfecv.fit(X, tf.squeeze(np.asarray(Y),1))

In [ ]:
print("Optimal number of features : %d" % rfecv.n_features_)

# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(min_features_to_select,
               len(rfecv.grid_scores_) + min_features_to_select),
         rfecv.grid_scores_)
plt.show()

<img src="https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Series_Temporelles/Multi/images/ResultRFECV_Nasdaq.png?raw=true" width=400>

In [ ]:
rfecv.ranking_

<img src="https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Series_Temporelles/Multi/images/ResultRFECV_Nasdaq2.png?raw=true" width=700>

In [ ]:
selected = np.asarray([66,1,9,1,50,1,42,16,6,15,10,55,1,1,1,1,19,21,56,36,53,7,62,2,29,40,35,63,1,60,17,32,30,4,61,68,33,1,12,38,28,1,54,22,49,41,3,44,1,58,5,48,64,57,27,13,67,1,31,37,24,11,20,8,69,59,43,51,26,52,65,47,18,34,45,39,25,46,14,1,23])
df_selected = pd.DataFrame()

for i in range(len(X.columns)):
  if selected[i] == 1:
    df_selected[X.columns[i]] = X[X.columns[i]]

df_selected

**Corrélation entre les variables**

In [ ]:
import seaborn as sns

corr = df_selected.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=np.bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(15, 8))

sns.heatmap(corr,mask=mask, cmap='coolwarm',annot=True, fmt='.2f')

**Exportation du dataframe :**

In [ ]:
cible = Y['NDX']
df_selected.insert(len(df_selected.columns),"NDX",cible)

In [ ]:
df_selected

In [ ]:
from google.colab import files

df_selected.to_csv("NASDAQ_RFE-CV.csv")
files.download("NASDAQ_RFE-CV.csv")

#Extraction des données pour VSURF

In [ ]:
X = df_etude.iloc[:,:-1]
X

In [ ]:
Y = df_etude.iloc[:,-1:]
Y

In [ ]:
from google.colab import files

X.to_csv("XNAS.csv")
Y.to_csv("YNAS.csv")

files.download("XNAS.csv")
files.download("YNAS.csv")

**2. Résultats**

Résultats :

> print(thres_nasdaq$varselect.thres)
 [1]  2 49  6 80 67 38 39 63 11 13  4 29 81 24 50  5 47 12 41 56 21  3 26 73 16 34 45 36  9 40
[31] 44 66 14 59 17 51 30 20 35 58 60 72 42 78 33 19 55 18 64 70 15 53 10 69 77 43 32 25 22 61
[61] 27  1 62 76  7 46  8 68 75 48 23 31 52 57 54 74 28 37 71 79 65

> print(interp_nasdaq$varselect.interp)
 [1]  2 49  6 80 67 38 39 63 11 13  4 29 81 24 50  5 47 12 41 56 21  3 26 73 16 34 45 36  9 40
[31] 44 66 14 59 17 51 30 20 35 58 60 72 42 78 33 19 55 18 64 70 15 53 10 69 77 43 32 25 22 61
[61] 27  1 62 76  7 46  8 68 75 48 23 31 52

**3. Création du dataframe**

In [ ]:
selected_pred = np.asarray([2,49,6,80,67,38,39,63,11,13,4,29,81,24,50,5,47,12,41,56,21,3,26,73,16,34,45,36,9,40,44,66,14,59,17,51,30,20,35,58,60,72,42,78,33,19,55,18,64,70,15,53,10,69,77,43,32,25,22,61,27,1,62,76,7,46,8,68,75,48,23,31,52])
selected_VSURF_pred = pd.DataFrame()

for i in range(len(selected_pred)):
    selected_VSURF_pred[X.columns[selected_pred[i]-1]] = X[X.columns[selected_pred[i]-1]]

selected_VSURF_pred['NDX'] = Y['NDX']
selected_VSURF_pred

In [ ]:
from google.colab import files

selected_VSURF_pred.to_csv("NASDAQselected_VSURF_pred.csv")

files.download("NASDAQselected_VSURF_pred.csv")


**Corrélation entre les variables**

In [ ]:
import seaborn as sns

corr = selected_VSURF_pred.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=np.bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(15, 8))

sns.heatmap(corr,mask=mask, cmap='coolwarm',annot=True, fmt='.2f')